# Renderer

In [22]:
import os
import sys
sys.path.append("..")
from glob import glob

from src.config.config import cfg

In [23]:
import numpy as np

## Renderer Class

In [24]:
class Renderer(object):
    intrinsic_matrix = {
        'linemod': np.array([[572.4114, 0., 325.2611],
                                                  [0., 573.57043, 242.04899],
                                                  [0., 0., 1.]]),
        # 'blender': np.array([[280.0, 0.0, 128.0],
        #                      [0.0, 280.0, 128.0],
        #                      [0.0, 0.0, 1.0]]),
        'blender': np.array([[700.,    0.,  320.],
                                                  [0.,  700.,  240.],
                                                  [0.,    0.,    1.]])
    }

    def __init__(self, object_name: str):
        """
        Renderer の初期化

        Arg:
            object_name (str): 対象とするオブジェクト名
        """
        self.object_name = object_name
        self.linemod_dir = cfg.LINEMOD_DIR
        self.pvnet_linemod_dir = cfg.PVNET_LINEMOD_DIR
        self.parent_dir_path = cfg.DATA_DIR

        self.bg_imgs_path = os.path.join(self.parent_dir_path, 'bg_imgs.npy')
        self.poses_path = os.path.join(self.parent_dir_path, 'blender_poses', '{}_poses.npy').format(object_name)
        self.output_dir_path = os.path.join(self.pvnet_linemod_dir,'renders/{}').format(object_name)
        self.blender_path = cfg.BLENDER_PATH
        self.blank_blend = os.path.join(self.parent_dir_path, 'blank.blend')
        self.py_path = os.path.join(cfg.BLENDER_DIR, 'render_backend.py')
        self.obj_path = os.path.join(self.pvnet_linemod_dir,'{}/{}.ply').format(object_name, object_name)
        self.plane_height_path = os.path.join(self.parent_dir_path, 'plane_height.pkl')

    def get_bg_imgs(self):
        if os.path.exists(self.bg_imgs_path):
            return

        img_paths = glob(os.path.join(cfg.SUN, 'JPEGImages/*'))
        bg_imgs = []

        for img_path in img_paths:
            img = Image.open(img_path)
            row, col = img.size
            if row > 500 and col > 500:
                bg_imgs.append(img_path)

        np.save(self.bg_imgs_path, bg_imgs)

    def project_model(self, model_3d, pose, camera_type):
        camera_model_2d = np.dot(model_3d, pose[:, :3].T) + pose[:, 3]
        camera_model_2d = np.dot(camera_model_2d, self.intrinsic_matrix[camera_type].T)
        return camera_model_2d[:, :2] / camera_model_2d[:, 2:]

    @staticmethod
    def exr_to_png(exr_path):
        depth_path = exr_path.replace('.png0001.exr', '.png')
        exr_image = OpenEXR.InputFile(exr_path)
        dw = exr_image.header()['dataWindow']
        (width, height) = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

        def read_exr(s, width, height):
            mat = np.fromstring(s, dtype=np.float32)
            mat = mat.reshape(height, width)
            return mat

        dmap, _, _ = [read_exr(s, width, height) for s in exr_image.channels('BGR', Imath.PixelType(Imath.PixelType.FLOAT))]
        dmap = Image.fromarray((dmap != 1).astype(np.int32))
        dmap.save(depth_path)
        exr_image.close()
        os.system('rm {}'.format(exr_path))

    def sample_poses(self):
        statistician = DataStatistics(self.object_name)
        statistician.sample_poses()

    def get_plane_height(self):
        if os.path.exists(self.plane_height_path):
            plane_height = read_pickle(self.plane_height_path)
        else:
            plane_height = {}

        if self.object_name in plane_height:
            return plane_height[self.object_name]
        else:
            pose_transformer = PoseTransformer(self.object_name)
            model = pose_transformer.get_blender_model()
            height = np.min(model[:, -1])
            plane_height[self.object_name] = height
            save_pickle(plane_height, self.plane_height_path)
            return height

    def run(self):
        """ Render images
        1. prepare background images
        2. sample poses from the pose distribution of training data
        3. call the blender to render images
        """
        self.get_bg_imgs()
        self.sample_poses()

        if not os.path.exists(self.output_dir_path):
            os.makedirs(self.output_dir_path)

        os.system('{} {} --background --python {} -- --input {} --output_dir {} --bg_imgs {} --poses_path {}'.
                  format(self.blender_path, self.blank_blend, self.py_path, self.obj_path,
                         self.output_dir_path, self.bg_imgs_path, self.poses_path))
        depth_paths = glob(os.path.join(self.output_dir_path, '*.exr'))
        for depth_path in depth_paths:
            self.exr_to_png(depth_path)

    @staticmethod
    def multi_thread_render():
        # objects = ['ape', 'benchvise', 'bowl', 'can', 'cat', 'cup', 'driller', 'duck',
        #            'glue', 'holepuncher', 'iron', 'lamp', 'phone', 'cam', 'eggbox']
        objects = ['lamp', 'phone']

        def render(object_name):
            renderer = Renderer(object_name)
            renderer.run()

        with Pool(processes=2) as pool:
            pool.map(render, objects)

## インスタンス生成

In [25]:
object_name = 'ape'

renderer=Renderer(object_name=object_name)

In [26]:
renderer.run()

AttributeError: 'EasyDict' object has no attribute 'SUN'